In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
use role accountadmin

In [ ]:
session.sql("""CREATE OR REPLACE DATABASE sales_db;""").collect()
session.sql("""CREATE OR REPLACE WAREHOUSE sales_wh
        WITH WAREHOUSE_SIZE = 'MEDIUM'
        AUTO_SUSPEND = 300
        AUTO_RESUME = TRUE""").collect()



In [ ]:
USE WAREHOUSE sales_wh;
USE DATABASE sales_db;
USE SCHEMA public;

In [ ]:
# Execute SQL command to create the table
session.sql("""
    CREATE OR REPLACE TABLE sales (
        sale_id INT AUTOINCREMENT PRIMARY KEY,
        sale_date DATE,
        amount DECIMAL(10, 2),
        region VARCHAR(50)
    );
""").collect()


In [ ]:
# Execute SQL command to insert sample data
session.sql("""
    INSERT INTO sales (sale_date, amount, region) VALUES
    ('2024-01-01', 150.00, 'North'),
    ('2024-01-02', 200.00, 'South'),
    ('2024-01-03', 250.00, 'East'),
    ('2024-01-04', 300.00, 'West'),
    ('2024-01-05', 350.00, 'North'),
    ('2024-01-06', 400.00, 'South');
""").collect()


In [ ]:
# Execute SQL command to create a transformed view
session.sql("""
    CREATE OR REPLACE VIEW sales_summary AS
    SELECT
        region,
        COUNT(sale_id) AS total_sales,
        SUM(amount) AS total_amount
    FROM sales
    GROUP BY region;
""").collect()


In [ ]:
import matplotlib.pyplot as plt

# Fetch data from the view
df = session.table("sales_summary").toPandas()

# Data visualization
plt.figure(figsize=(10, 6))
df.plot(kind='bar', x='REGION', y='TOTAL_AMOUNT', legend=False)
plt.title('Total Sales by Region')
plt.xlabel('Region')
plt.ylabel('Sales Amount')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
CREATE OR REPLACE STAGE TELCO_CHURN_EXTERNAL_STAGE_DEMO
    URL = 's3://sfquickstarts/notebook_demos/churn/' 

In [ ]:
CREATE FILE FORMAT IF NOT EXISTS MY_PARQUET_FORMAT TYPE = PARQUET COMPRESSION = SNAPPY;

In [ ]:
CREATE TABLE if not exists TELCO_CHURN_RAW_DATA_DEMO USING TEMPLATE ( 
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) 
    FROM 
        TABLE( INFER_SCHEMA( 
        LOCATION => '@TELCO_CHURN_EXTERNAL_STAGE_DEMO', 
        FILE_FORMAT => 'MY_PARQUET_FORMAT',
        FILES => 'telco_churn.parquet'
        ) 
    ) 
);

In [ ]:
COPY INTO TELCO_CHURN_RAW_DATA_DEMO
FROM @TELCO_CHURN_EXTERNAL_STAGE_DEMO
FILES = ('telco_churn.parquet')
FILE_FORMAT = (
    TYPE=PARQUET,
    REPLACE_INVALID_CHARACTERS=TRUE,
    BINARY_AS_TEXT=FALSE
)
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
ON_ERROR=ABORT_STATEMENT;